
# Trabalho 2 - Tópicos em IA 
## (**Parte 2**)

### Realizando Predições e Visualizando Métricas
---


## Introdução

Neste ponto, os modelos foram treinados e estão salvos em /documents/bertimbau-(baseline|alternativo)-cls-ptbr
Vamos carregá-los, realizar as predições e visualizar métricas

**Observação**: Para realizar as predições, precisamos do `test_dataset`, que criamos na primeira parte do trabalho.

Para isso, podemos replicar o código aqui, ou salvar e pegar apenas ele. Salvei na pasta documents, que contém o corpus de notícias e os modelos. Para replicar este notebook, os passos do notebook anterior devem ser replicados desde o começo, e o corpus de notícia deve estar corretamente rastreado (ou, quem sabe, no futuro, pegar o modelo direto do meu hugging face).


---

## Configs Iniciais

#### configurações iniciais do path

In [ ]:

import os, sys
root = os.path.abspath('../')
src_path = os.path.join(root, 'src')
sys.path.append(root)

#### imports

In [ ]:
from safetensors.torch import load_file

from torch import load as t_load
from src.custom_bertimbau_classifier import CustomBertimbauClassifier
from src.baseline_bertimbau_classifier import BaselineBertimbauClassifier
from src.dataset_loader import DatasetLoader, FakeNewsDataset
import src.config as cfg

from transformers import (
    BertForSequenceClassification,
    Trainer, TrainingArguments
    )

## Carregando Trainer e Dataset para Predição

### load de `test_dataset`
Reproduzindo o código de antes para capturar test_dataset e o tokenizer

In [ ]:
path = os.path.join(root, cfg.PATH_TO_DATASET)
ds_loader = DatasetLoader(
    path=path,
    model_name=cfg.BERTIMBAU,
    max_len=cfg.SEQ_LEN
)

#conjunto para teste, validação e treino
_, _, test_dataset = (ds_loader
                        .load_dataset(seed=cfg.RANDOM_SEED)
                        .get_datasets())

tokenizer = ds_loader.get_tokenizer()

len(test_dataset)


### Load do Trainer
Para isso, aproveitaremos dos arquivos salvos devido à configuração `save_strategy`, de `TrainingArguments`

#### load dos models

In [ ]:

#algumas funções simples pra simplificar o script
get_checkpoint_dir = lambda base_alt: os.path.join(root,"documents",f"bertimbau-{base_alt}-cls-ptbr","checkpoint-630") 
get_state_dict = lambda base_alt: load_file(os.path.join(get_checkpoint_dir(base_alt),"model.safetensors")) 

# print(type(get_checkpoint_dir("baseline")))

In [ ]:

tensor_dict = get_state_dict("baseline")
baseline_model = BertForSequenceClassification.from_pretrained(state_dict=tensor_dict, num_labels=2)

custom_tensor_dict = get_state_dict("alternativo")
# custom_model = CustomBertimbauClassifier

In [ ]:

baseline_checkpoint_dir =  get_checkpoint_dir("baseline")
tokenizer = ds_loader.get_tokenizer()
model_kwargs = ds_loader.get_labels_mapping()

baseline_model = BertForSequenceClassification.from_pretrained(
    baseline_checkpoint_dir,
    **model_kwargs
).to(cfg.DEVICE)

# baseline_training_args = TrainingArguments.from_pretrained(baseline_checkpoint_dir)
baseline_training_args = t_load(os.path.join(baseline_checkpoint_dir, "training_args.bin"))
assert isinstance(baseline_training_args, TrainingArguments)

baseline_trainer = Trainer(
    model=baseline_model,
    args=baseline_training_args,
    tokenizer=tokenizer,
)

baseline_trainer.predict(test_dataset)

In [ ]:
# baseline_trainer = Trainer(
#     )

# baseline_trainer.train(
#     resume_from_checkpoint=os.path.join(root,"documents","bertimbau-baseline-cls-ptbr","checkpoint-630"))

# preds = baseline_trainer.predict(test_dataset)
# preds.metrics

## Realizando Predições